First install the libraries needed for the detection:

In [ ]:
%pip install opencv-python opencv-python-headless numpy ultralytics

Text to speech library:

In [ ]:
%pip install pyttsx3

Speech recognision:

In [ ]:
%pip install SpeechRecognition

In [ ]:
%pip install pyaudio

For debugging purposes, Isn't needed for normal use:

In [ ]:
%pip install -U ipykernel

All the pre-trained classes in the order of their indexes

In [9]:
COCO_CLASSES = [
    "person", "bicycle", "car", "motorbike", "airplane", "bus", "train", "truck", "boat",
    "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
    "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack",
    "umbrella", "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball",
    "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", "tennis racket",
    "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple",
    "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair",
    "couch", "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse",
    "remote", "keyboard", "cell phone", "microwave", "oven", "toaster", "sink", "refrigerator",
    "book", "clock", "vase", "scissors", "teddy bear", "hair drier", "toothbrush"
]

FOCAL_LENGHT = 1401.191 # 
YOLO_MODEL = "yolov8n.pt"
WIDTH_CAMERA = 600

    

def get_class_name(id):
    return COCO_CLASSES[id]

Imports:

In [2]:
import tkinter as tk
from PIL import Image, ImageTk
import cv2
from ultralytics.models import YOLO
from ultralytics import solutions, YOLO
import pyttsx3
import speech_recognition as sr
from threading import Thread

Starts the model and camera input (needed to run before the main loop each time):

In [ ]:
# Load the YOLOv8 model (pre-trained on COCO dataset)
model = YOLO(YOLO_MODEL)  # Use yolov8n.pt for a lightweight model

distance_calc = solutions.distance_calculation.DistanceCalculation(model= YOLO_MODEL, show=True)

# Initialize the webcam
cap = cv2.VideoCapture(0)



Sound recognition code:

In [ ]:
# Text To Speech initialization
tts = pyttsx3.init()
tts.setProperty("rate", 150)
tts.setProperty("volume", 0.9)

bool_command_given = True
bool_wait_for_response = False

user_input = "*"

global searched_class

searched_class = "laptop" # The name of the class needs to be one from the COCO_CLASSES, this is the class that is searched by the camera

enable_speech = False


def get_speech():
    recognizer = sr.Recognizer()
    mic = sr.Microphone()
    
    with mic as source:
        recognizer.adjust_for_ambient_noise(source)
        sound = recognizer.listen(source)
        
    try:
        searched_class = recognizer.recognize_tensorflow(
            sound, 
            "speech_commands_v0.01/conv_action_frozen.pb", # Trained model for speech to text
            "speech_commands_v0.01/conv_actions_labels.txt").lower()
        
        
        if searched_class in [COCO_CLASSES]:
            tts.say("Searching for {searched_class}")
            tts.runAndWait()
        else:
            tts.say("Command didn't understood")
            tts.runAndWait()
            
    except sr.UnknownValueError:
        print("Sorry, I didn't catch that.")
    except sr.RequestError:
        print("Could not request results. Please check your network connection.")
        
    return searched_class
    

## Main Loop:

In [ ]:
# Create the main Tkinter window
window = tk.Tk()
window.title("YOLOv8 Detection")
window.geometry("800x600")

# Create a canvas to display the video feed
canvas = tk.Canvas(window, width=640, height=480)
canvas.pack()

def read_class(class_name):
    tts.say(class_name)
    tts.runAndWait()    

def calc_position(x1, x2):
    if x1 > 200:
        return "to the right"
    elif x2 < 400:
        return "to the left"
    else:
        return "in front of you"
    
def update_frame():
    global searched_class
    if enable_speech:
        user_input = get_speech()
        if user_input in "reset":
            bool_command_given = True
            bool_wait_for_response = True
            
        elif user_input in "stop":
            searched_class = "$"
        
        if bool_wait_for_response and user_input.split() in COCO_CLASSES:
            searched_class = user_input
            bool_wait_for_response = False

    ret, frame = cap.read()
    if ret:
        # Run YOLOv8 on the frame
        results = model.predict(source=frame, conf=0.6, show=False)

        # Detected Classes
        for result in results[0].boxes:
            # Gets the id of detected class
            class_id = int(result.cls[0])
            class_name = get_class_name(class_id)
            confidence = result.conf[0]
            x1, y1, x2, y2 = result.xyxy[0].cpu().numpy()
            
            # frame = distance_calc.calculate(frame)
            
            print("x1:", x1, "x2:", x2)

            if class_name in searched_class:
                label = f"{class_name}: {confidence:.4f}"
                # Draw a bounding box and label
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 2)
                
                read_class(calc_position(x1, x2))
                
                

        # Convert the frame to RGB format for Tkinter
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame_rgb)
        imgtk = ImageTk.PhotoImage(image=img)

        # Update the canvas with the new frame
        canvas.create_image(0, 0, anchor=tk.NW, image=imgtk)
        canvas.imgtk = imgtk

    # Repeat the function after a delay
    window.after(10, update_frame)

def on_closing():
    # Release the video capture and destroy the Tkinter window
    cap.release()
    window.destroy()
if enable_speech:
    listen_thread = Thread(target=get_speech, daemon=True)
    listen_thread.start()

# Start the frame update loop
update_frame()

# Set up the window close event
window.protocol("WM_DELETE_WINDOW", on_closing)

# Run the Tkinter main loop
window.mainloop()